# DS-GA-3001 Advanced Python for Data Science

Before you turn this problem in, make sure you **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart). You can then run the cells **in order**, during the class.

Any textual answers that need to be provided will be marked with "YOUR ANSWER HERE". Replace this text with your answer to the question.

Any code answers that need to be provided will be marked with:

```
# YOUR CODE HERE
raise NotImplementedError()
```

Replace all this code with your answer to the question. If you do not answer the question, the `NotImplementedError` exception will be raised, which will indicate to the grader that no answer has been supplied.

In many cases, code answers will also have some associated test code. You should execute the tests after you have entered your code in order to ensure that your answer is correct. You should not proceed to the next question until your answer is correct.

Finally, insert your Net ID and the Net ID's of any collaborators in the cell below.

In [1]:
NET_ID = "jl6583"
COLLABORATORS = ""

---

# Introduction to Parallel Programming with MPI (Part 1)

## Setting up Jupyter for MPI

### 1. Make sure ipyparallel is installed

    conda install ipyparallel

### 2. Create jupyter profile

    jupyter notebook --generate-config
    
### 3. Add ipyparallel extension
    
Edit the `jupyter_notebook_config.py` file

    nano ~/.jupyter/jupyter_notebook_config.py
    
Add the following line anywhere:

    c.NotebookApp.server_extensions.append('ipyparallel.nbextension')
    
Save the file and exit.

Edit the `jupyter_notebook_config.json` file (create a new one if it doesn't exist):

    nano ~/.jupyter/jupyter_notebook_config.json
    
Add the following line:

    {"NotebookApp": {"server_extensions": ["ipyparallel.nbextension"]}}
    
Save the file and exit.


*Note: if `jupyter_notebook_config.json` already exists, find the `"server_extensions"` entry and add a comma to the end of the last extension and `"ipyparallel.nbextension"` after that.*

### 4. Enable the IPython Clusters tab in the Jupyter Notebook

    ipcluster nbextension enable
    
### 5. Create an MPI profile

    ipython profile create --parallel --profile=mpi
    
### 6. Add MPI to the profile

    nano ~/.ipython/profile_mpi/ipcluster_config.py
    
Add the following line anywhere:

    c.IPClusterEngines.engine_launcher_class = 'MPIEngineSetLauncher'
    
Save the file and exit.

### 7. Start the engines

    ipcluster start -n 4 --profile=mpi

### 8. Restart jupyter notebook

Stop the jupyter notebook server, then start a new one.

    jupyter notebook
    
Check that the **IPython Clusters** tab is visible. Switch to this tab and check and there is an **mpi** entry on the tab. Press the refresh button if not:

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgamVKb1cyU210S0k)

### 9. Start the MPI cluster

On the **mpi** entry, set the **# of engines** to **4**, then click the **Start** button. 

***Note that the # of engines must be less than or equal to the number of engines specified in step 7.***

### 10. Initialize MPI

Run the following code to initialize MPI. Make sure there are no errors. 

*** Note: You may need to wait a few seconds after starting the engines before this code will work. Try running it again if you get an error.***

In [1]:
%pylab inline
from ipyparallel import Client, error
cluster = Client(profile="mpi")
view = cluster[:]
view.block=True

Populating the interactive namespace from numpy and matplotlib


## Definitions

**Program** - an executable file residing on disk

**Process (or task)** - one or more executing instances of a program. Processes have separate address spaces.

**Thread (or lightweight process)** - one or more threads of control within a process. Threads share the same address space.

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgSFNteVVKaXhCLTg)

## What is parallel computing?

Traditionally, software has been written for ***serial*** computation:
* A problem is broken into a discrete series of instructions
* Instructions are executed sequentially one after another
* Executed on a single processor
* Only one instruction may execute at any moment in time

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgcXNYZWtfMHpBdk0)

In the simplest sense, ***parallel computing*** is the simultaneous use of multiple compute resources to solve a computational problem:
* A problem is broken into discrete parts that can be solved concurrently
* Each part is further broken down to a series of instructions
* Instructions from each part execute simultaneously on different processors
* An overall control/coordination mechanism is employed

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgUUNKZUdtQnRISEU)

## Why do we need parallel programming?

Need to solve larger problems
* Require more memory
* Computation takes *much* longer
* Huge amounts of data may be required

Parallel programming provides
* More CPU resources
* More memory resources
* Solve problems that were not possible with serial program
* Solve problems more quickly

## Two basic approaches

### Shared Memory Computer
* Used by most laptops/PCs
* Multiple cores (CPUs)
* Share a global memory space
* Cores can efficiently exchange/share data

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgaGhfOVo1S0pMT3c)

### Distributed Memory (ex. Compute cluster)
* Collection of nodes which have multiple cores
* Each node uses its own local memory
* Work together to solve a problem
* Communicate between nodes and cores via messages
* Nodes are networked together

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgU2VBMVFPS2hVdDA)

## Parallel programming models

Directives-based parallel programming language 
* OpenMP (most widely used)
* High Performance Fortran (HPF) is another example
* Directives tell processor how to distribute data and work across the processors
* Directives appear as comments in the serial code
* Implemented on shared memory architectures

Message Passing
* MPI (most widely used)
* Pass messages to send/receive data between processes
* Each process has its own local variables
* Can be used on either shared or distributed memory architectures

## Pros and cons

Pros of OpenMP
* Easier to program and debug than MPI
* Directives can be added incrementally - gradual parallelization
* Can still run the program as a serial code
* Serial code statements usually don't need modification
* Code is easier to understand and maybe more easily maintained

Cons of OpenMP
* Can only be run in shared memory computers 
* Requires a compiler that supports OpenMP
* Mostly used for loop parallelization 

Pros of MPI
* Runs on either shared or distributed memory architectures
* Can be used on a wider range of problems than OpenMP
* Each process has its own local variables
* Distributed memory computers are less expensive than large shared memory computers

Cons of MPI
* Requires more programming changes to go from serial to parallel version
* Can be harder to debug
* Performance is limited by the communcation network between the nodes

## Parallel programming issues

Goal is to reduce execution time
* Computation time
* Idle time - waiting for data from other processors
* Communication time - time the processors take to send and receive messages

Load Balancing 
* Divide the work equally among the available processors

Minimizing Communication
* Reduce the number of messages passed 
* Reduce amount of data passed in messages

Where possible - overlap communication and computation

Many problems scale well to only a limited number of processors

## Amdahl's law

$\large\displaystyle{speedup=\frac{1}{(1-P)+\frac{P}{n}}}$

where

* $speedup$ is the theoretical speedup of the execution of the whole program
* $n$ is the number of parallel threads/processes
* $P$ is the fraction of the algorithm that can be made parallel 

Basically this is saying that the amount of speedup a program will see by using $n$ cores is based on how much of the program is serial (can only be run on a single CPU core) and how much of it is parallel (can be split up among multiple CPU cores).

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgdXl1Z05reU1aUHc)

## Programming approaches

Two main approaches:

SPMD - Single Program, Multiple Data Streams
* Each processor is executing the same program on different data
* A parallel execution model that assumes multiple cooperating processes executing a program
* The most common style of parallel programming and the one used by MPI

MPMD - Multiple Programs, Multiple Data Streams
* Multiple processors executing at least two independent programs
* Manager/worker strategies fit into this category
* Web browser and web server is another example

## What is MPI?

MPI stands for Message Passing Interface
* Library of functions (C/C++) or subroutines (Fortran)

History
* Early message passing Argonne's P4 and Oak Ridge PVM in 1980s
* MPI-1 completed in 1994 (1.1 - 1995, 1.2 - ?, 1.3 - 2008)
* MPI-2 completed in 1998 (2.1 - 2008, 2.2 - 2009)
* MPI-3 completed in 2012 (3.1 - 2015)

MPI-3 features gradually added to MPI implementations

## Version differences

Examples of  Different Implementations
* MPICH - developed by Argonne Nationa Labs (freeware)
* MPI/LAM  - developed by Indiana, OSC, Notre Dame (freeware)
* MPI/Pro - commerical product
* Apple's X Grid 
* OpenMPI - MPI-2 compliant, thread safe

Similiarities in Various Implementations
* Source code compatibility (except parallel I/O)
* Programs should compile and run as is
* Support for heterogeneous parallel architectures

Difference in Various Implementations
* Commands for compiling and linking
* How to launch an MPI program
* Parallel I/O (from MPI-2)
* Debugging

## Message passing (MPI)

The most commonly used method of programming distributed-memory MIMD systems is message passing, or some variant of message passing. MPI is the most widely used standard.




In basic message passing, the processes coordinate their activities by explicitly sending and receiving messages. Explicit sending and receiving messages is known as ***point to point*** communication.



MPI's send and receive calls operate in the following manner:
1. First, process A decides a message needs to be sent to process B. 
2. Process A then packs up all of its necessary data into a buffer for process B. 
3. Process A indicates that the data should be sent to process B by calling the `Send` function. 
4. Before process B can receive the data, it needs to acknowledge that it wants to receive it. Process B does this by calling the `Recv` function.

In this way, every time a process sends a message, there must be a process that also indicates it wants to receive the message. i.e. calls to `Send` and `Recv` are always paired.


![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgNVNacS1sdzlPbWc)

### How does a process know where to send a message?

The number of processes is fixed when an MPI program is first started (there is a way to create more processes, but we will ignore that for now.) Each of the processes is assigned a unique integer starting from 0. This integer is know as the *rank* of the process and is how each process is identified when sending and receiving messages.

MPI processes are arranged in logical collections that define which processes are allowed to send and receive messages. A collection of this type is known as *communicator*. Communicators can be arranged in an hierarchy, but as this is seldom used in MPI, we will not consider it more here. There is one special communicator that exists when an MPI program starts, that contains all the processes in the MPI program. This communicator is called `MPI.COMM_WORLD`. In `mpi4py`, communicators are represented by the `Comm` class.


In order for a process to learn about other processes, MPI provides two methods on a communicator. The first of these is called `Get_size()`, and this returns the total number of processes contained in the communicator (the *size* of the communicator). The second of these is called `Get_rank()`, and this returns the rank of the calling process within the communicator. Note that `Get_rank()` will *return a different value* for every process in the MPI program.


For simplicity, we will refer to the "process who's rank is N" and "process N".


The following code obtains the size of the `MPI.COMM_WORLD` communicator, and rank of the process within the communicator. Run this code to see what are the values of `size` and `rank` for each process.

In [2]:
%%px
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print 'size=%d, rank=%d' % (size, rank)

[stdout:0] size=4, rank=1
[stdout:1] size=4, rank=2
[stdout:2] size=4, rank=0
[stdout:3] size=4, rank=3


<div class="alert alert-success">
What do you notice about the order that the program prints the values in? *Hint: try running the program a few times.*
</div>

The order the values are printed is fixed

## One MPI program, multiple MPI processes

When an MPI program is run, each process consists of the same code. However, as we've seen, each process is assigned a different rank. This allows code for each process to be embedded within one program file. 

In the following code, all processes start with the same two numbers `a` and `b`. However, although there is only one file, each process performs a different computation on the numbers. Process 0 prints the sum of the numbers, process 1 prints the result of multiplying the numbers, and process 2 prints the maximum value.

In [3]:
%%px
from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 6.0
b = 3.0
if rank == 0:
        print a + b
if rank == 1:
        print a * b
if rank == 2:
        print max(a,b)

[stdout:0] 18.0
[stdout:1] 6.0
[stdout:2] 9.0


<div class="alert alert-success">
Write a program in which the processes with even rank print "Hello" and the processes with odd rank print "Goodbye". Print the rank along with the message (for example "Goodbye from process 3"). *Hint: remember that although the number of processes is fixed when the program startes, the exact number is not known until the Get_size() method is called.*
</div>

In [4]:
%%px
rank = MPI.COMM_WORLD.Get_rank()
if rank % 2 == 0:
    print 'rank=',rank, 'Hello'
else:
    print 'rank=',rank,'Goodbye'

[stdout:0] rank= 1 Goodbye
[stdout:1] rank= 2 Hello
[stdout:2] rank= 0 Hello
[stdout:3] rank= 3 Goodbye


## Point-to-point communication

As mentioned in earlier, the simplest message passing involves two processes: a sender and a receiver. Let us begin by demonstrating a program designed for two processes. One will draw a random number and then send it to the other. We will do this using the routines `Send` and `Recv`.

In [5]:
%%px
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) #different process has different local variable space

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print "Process", rank, "drew the number", randNum[0]
        comm.Send(randNum, dest=0)

if rank == 0:
        print "Process", rank, "before receiving has the number", randNum[0]
        comm.Recv(randNum, source=1)
        print "Process", rank, "received the number", randNum[0]

[stdout:0] Process 1 drew the number 0.462338781739
[stdout:2] 
Process 0 before receiving has the number 0.0
Process 0 received the number 0.462338781739


The `Send` and `Recv` functions are referred to as *blocking* functions. If a process calls `Recv` it will simply wait until a message from the corresponding `Send` in received before proceeding. Similarly the `Send` will wait until the message has been reveived by the corresponding `Recv`.

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgOFJmOVA2Z2JycTQ)

### Deadlock

Because `Send` and `Recv` are blocking functions, a very common situation that can occur is called ***deadlock***. This happens when one process is waiting for a message that is never sent. We can see a simple example of this by commenting out the `comm.Send` and running the program below.

In [ ]:
%%px
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print "Process", rank, "drew the number", randNum[0]
        #comm.Send(randNum, dest=0)

if rank == 0:
        print "Process", rank, "before receiving has the number", randNum[0]
        comm.Recv(randNum, source=1)
        print "Process", rank, "received the number", randNum[0]

<div class="alert alert-danger">
When you run this code, it will hang. In fact, the Jupyter kernel is now locked up, so that if you try to run any other code it will also hang. 
</div>

When this happens, you will need to do the following:
1. Open the **IPython Clusters** tab and click on the **Stop** button
2. Stop the **ipcluster** command (usually Control-C)
3. Stop the **jupyter notebook** command (usually Control-C)
4. Close this notebook.
5. Rerun **jupyter notebook**
6. Rerun **ipcluster start -n 4 --profile=mpi**
7. Restart 4 engines on the **IPython Clusters** tab.

You will also need to re-run the initialization code again (which has been copied below for convenience).

In [6]:
%pylab inline
from ipyparallel import Client, error
cluster = Client(profile="mpi")
view = cluster[:]
view.block=True

Populating the interactive namespace from numpy and matplotlib


### More Send and Recv

Previously we saw how to send a message from one process to another. Now we're going to try sending a message to a process and receiving a message back again.

<div class="alert alert-success">
Modify the previous (working) code so that when the process 0 receives the number, it multiplies it by two and sends it back to process 1. Process 1 should then print out the new value.
</div>

![](http://drive.google.com/uc?export=view&id=0B_3lImS7uRMgS0FoSGFvVnRRN3c)

In [7]:
%%px
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print "Process", rank, "drew the number", randNum[0]
        comm.Send(randNum, dest=0)
        #process 1 tries to receive the number back from process 0
        comm.Recv(randNum, source=0)
        print 'Process', rank, 'received the multiplied number', randNum[0]
        

if rank == 0:
        print "Process", rank, "before receiving has the number", randNum[0]
        comm.Recv(randNum, source=1)
        print "Process", rank, "received the number", randNum[0]
        #process 0 multiplies the number and send it back to 1
        comm.Send(randNum*2, dest=1)
        print 'Process', rank, 'Send back the multiplied number'

[stdout:0] 
Process 1 drew the number 0.393592804058
Process 1 received the multiplied number 0.787185608116
[stdout:2] 
Process 0 before receiving has the number 0.0
Process 0 received the number 0.393592804058
Process 0 Send back the multiplied number


The receiving process does not always need to specify the source when issuing a `Recv`. Instead, the process can accept any message that is being sent my another process. This is done by setting the source to `MPI.ANY_SOURCE`.

<div class="alert alert-success">
Try replacing the `source=N` arguments in your program with `source=MPI.ANY_SOURCE` to see if it still works.
</div>

In [8]:
%%px
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print "Process", rank, "drew the number", randNum[0]
        comm.Send(randNum, dest=0)
        #process 1 tries to receive the number back from process 0
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print 'Process', rank, 'received the multiplied number', randNum[0]
        

if rank == 0:
        print "Process", rank, "before receiving has the number", randNum[0]
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print "Process", rank, "received the number", randNum[0]
        #process 0 multiplies the number and send it back to 1
        comm.Send(randNum*2, dest=1)
        print 'Process', rank, 'Send back the multiplied number'

[stdout:0] 
Process 1 drew the number 0.21132575427
Process 1 received the multiplied number 0.422651508541
[stdout:2] 
Process 0 before receiving has the number 0.0
Process 0 received the number 0.21132575427
Process 0 Send back the multiplied number


### A Final word on the Send and Recv methods

<pre>Comm.<b>Send</b>(buf, dest=0, tag=0)</pre>

Performs a basic send. This send is a point-to-point communication. It sends information from exactly one process to exactly one other process.

Parameters:

* **Comm (MPI comm)** – communicator we wish to query
* **buf (choice)** – data to send
* **dest (integer)** – rank of destination
* **tag (integer)** – message tag


<pre>Comm.<b>Recv</b>(buf, source=0, tag=0, status=None)</pre>

Performs a point-to-point receive of data.

Parameters:

* **Comm (MPI comm)** – communicator we wish to query
* **buf (choice)** – initial address of receive buffer (choose receipt location)
* **source (integer)** – rank of source
* **tag (integer)** – message tag
* **status (Status)** - status of object

Sometimes there are cases when a process might have to send many different types of messages to another process. Instead of having to go through extra measures to differentiate all these messages, MPI allows senders and receivers to also specify message IDs (known as *tags*) with the message. The receiving process can then request a message with a certain tag number and messages with different tags will be buffered until the process requests them.